In [1]:
%matplotlib notebook
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point,LineString,  mapping;
from shapely.ops import cascaded_union
from spacetx_spatial_analysis import polyToGeoJson
import json
import geojson

from matplotlib import pyplot as plt

## Load cell x gene data from 1 dataset

In [ ]:
all_cells = pd.read_csv("/Users/brian/Downloads/mask_prior/segmentation_counts.tsv", delimiter = "\t")

In [ ]:
counts = all_cells.T
counts.columns = all_cells["gene"]
counts = counts[1:]

In [2]:
metadata = pd.read_csv("/Users/brian/Downloads/mask_prior/segmentation_cell_stats.csv")

In [3]:
metadata["x_um"] = 0.1*metadata["x"]
metadata["y_um"] = 0.1*metadata["y"]

In [4]:
v = napari.Viewer()

In [5]:
v.add_points(metadata[["x_um","y_um"]], size=10)

<Points layer 'Points' at 0x10ba845e0>

# load our annotated the layer locations 

In [77]:
with open("Allen_smFISH_annotations_geo.json",'r') as r:
    gcnew = geojson.load(r)

shapes_to_add = [np.array(geo["coordinates"][0]) for geo in gcnew["geometries"][1:-1]]
column_points = gcnew["geometries"][-1]["coordinates"]

annotation_names = [g['name'] for g in gcnew["geometries"][1:]]
v.add_shapes(shapes_to_add,shape_type='polygon',
    edge_width=1,
    edge_color='coral',
    face_color='royalblue',
    name="layer annotations")

# for ii,g in enumerate(shapes_to_add[:-1]):
#     carray = g
#     v.add_shapes([carray],shape_type='polygon',
#     edge_width=1,
#     edge_color='coral',
#     face_color='royalblue',
#     name=gcnew["geometries"][ii]["name"])


<Shapes layer 'layer annotations' at 0x155985af0>

In [78]:
annotation_names

['VISp_I', 'VISp_II/III', 'VISp_IV', 'VISp_V', 'VISp_VI', 'cortical_axis']

In [79]:


shapes_layer = v.layers[-1]

is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in v.layers[0].data] for ii,p in enumerate(shapes_layer.data)}
print(column_points)
print(is_in_layer.keys())

[[6895.070696, 5736.902356], [12811.78951, 15864.619246]]
dict_keys([0, 1, 2, 3, 4])


In [67]:
# depth coordinate calculation

def depth_coordinate(points_xy, column_axis_points):
    """ 
    calculate the projection along the column axis vector
    assumes that the pia is the first point in column_axis_points
    """
    column_vec = np.diff(column_axis_points, axis = 0)
    column_vec = column_vec.T/np.linalg.norm(column_vec,2)
    
    output_array = np.row_stack(
        [np.dot(a_point-column_axis_points[0],column_vec) for a_point in points_xy])
    return output_array

In [80]:
metadata["layer"] = "outside_VISp"
for l_key in is_in_layer.keys():
    metadata.loc[is_in_layer[l_key],"layer"] = annotation_names[l_key]
    

In [81]:
metadata["depth_um"] = depth_coordinate(metadata[["x_um","y_um"]].values, np.array(column_points)*.1)

In [82]:
metadata.loc[metadata["layer"]=="VISp_I"]["depth_um"].mean()

69.67768380830505

In [83]:
metadata.loc[metadata["layer"]=="VISp_II/III"]["depth_um"].mean()

280.8414099477571

In [84]:
metadata.loc[metadata["layer"]=="VISp_V"]["depth_um"].mean()

775.9941428386185

In [85]:
metadata.loc[metadata["layer"]=="VISp_VI"]["depth_um"].mean()

1043.1176974356924

In [88]:
metadata.to_csv("annotated_smFISH_cells.csv")

In [86]:
metadata["layer"].unique()

array(['VISp_VI', 'outside_VISp', 'VISp_IV', 'VISp_II/III', 'VISp_V',
       'VISp_I'], dtype=object)

In [87]:
metadata

,cell,x,y,has_center,cluster,n_transcripts,density,elongation,area,avg_confidence,doublet_score,x_um,y_um,layer,depth_um
0,1,9261.035253,15447.259475,False,4,125,0.009866,1.936,12670.0000,0.9748,0.003106,926.103525,1544.725947,VISp_VI,957.787655
1,2,6019.329058,20710.346863,False,2,104,0.009236,2.842,11260.0000,0.9818,0.485300,601.932906,2071.034686,outside_VISp,1248.705103
2,3,7741.731519,9911.947365,False,6,22,0.009594,4.348,2293.0000,0.9821,NaN,774.173152,991.194737,VISp_IV,403.202494
3,4,20329.880690,7044.657880,False,5,388,0.016530,2.669,23470.0000,0.9969,0.904800,2032.988069,704.465788,outside_VISp,790.618254
4,5,15485.972999,8216.108509,False,5,220,0.014860,2.119,14800.0000,0.9966,0.157100,1548.597300,821.610851,VISp_IV,647.422891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,4836,17390.912435,3267.370712,False,1,4,17.850000,26.530,0.2241,1.0000,NaN,1739.091244,326.737071,outside_VISp,316.216598
4836,4837,4325.081324,7962.837675,False,1,13,0.049020,1.184,265.2000,0.9811,NaN,432.508132,796.283768,VISp_I,62.558653
4837,4838,12581.642786,5113.917122,False,2,3,0.089660,11.780,33.4600,0.9947,NaN,1258.164279,511.391712,VISp_II/III,233.059585
4838,4839,9971.579329,7095.964142,False,1,10,3.384000,29.350,2.9550,1.0000,NaN,997.157933,709.596414,VISp_II/III,272.538244
